In [ ]:
import pandas as pd
import time
import random
from pytrends.request import TrendReq

# 1. AYARLAR
# ---------------------------------------------------------
# Retry mekanizmasını kapattık (method_whitelist hatası almamak için)
pytrends = TrendReq(hl='en-US', tz=360, retries=0, backoff_factor=0)

TIMEFRAME = 'today 5-y'

# DİKKAT: Threshold'u iptal ediyoruz. Ham veriyi görelim.
# THRESHOLD = 0 

keyword_map = [
    {
        "category": "FINANCE",
        "topic_title": "Economic Crisis", 
        "topic_id": "/g/1211cg58"           
    },
    {
        "category": "TEST", 
        "topic_title": "Depression(health)",
        "topic_id": "/m/02bft"            
    }
]

In [22]:
def fetch_trends_safe(topic_id, topic_title, retries=3):
    for attempt in range(retries):
        try:
            # Global çekim
            pytrends.build_payload(kw_list=[topic_id], timeframe=TIMEFRAME, geo='')
            df = pytrends.interest_by_region(resolution='COUNTRY', inc_low_vol=True, inc_geo_code=False)
            df = df.rename(columns={topic_id: topic_title})
            return df
        except Exception as e:
            wait_time = (attempt + 1) * 5 + random.randint(1, 5) # Bekleme süresini kısalttım test için
            print(f"⚠️ HATA ({topic_title}): {e}. Bekleniyor...")
            time.sleep(wait_time)
    return None

In [23]:
all_data_frames = []

print(f"🚀 Veri çekiliyor (Filtresiz)...")

for item in keyword_map:
    kw_title = item['topic_title']
    kw_id = item['topic_id']
    
    print(f"📡 {kw_title}...")
    df_temp = fetch_trends_safe(kw_id, kw_title)
    
    if df_temp is not None:
        # BURASI DEĞİŞTİ: Threshold kontrolü yapmadan direkt ekliyoruz.
        all_data_frames.append(df_temp)
    
    time.sleep(random.randint(2, 5))

🚀 Veri çekiliyor (Filtresiz)...
📡 Economic Crisis...
📡 Depression(health)...


In [24]:
if all_data_frames:
    final_df = pd.concat(all_data_frames, axis=1)
    final_df = final_df.fillna(0)
    
    # Sadece tamamen boş (tüm sütunları 0) olanları temizle
    # (Google Trends bazen hiç verisi olmayan adaları da listeler)
    final_df = final_df.loc[(final_df != 0).any(axis=1)]

    print(f"✅ Bitti. Boyut: {final_df.shape}")
    
    # Kontrol: Türkiye ve ABD var mı?
    check_countries = ['Türkiye', 'United States', 'China', 'Germany']
    print("\n--- Hedef Ülke Kontrolü ---")
    for country in check_countries:
        if country in final_df.index:
            print(f"✅ {country} Mevcut -> Skorları: {final_df.loc[country].values}")
        else:
            print(f"❌ {country} LİSTEDE YOK (Veri 0 geldiği için silinmiş olabilir)")

    final_df.to_csv("trends_data_no_filter.csv")
else:
    print("Veri yok.")

✅ Bitti. Boyut: (226, 2)

--- Hedef Ülke Kontrolü ---
✅ Türkiye Mevcut -> Skorları: [3 9]
✅ United States Mevcut -> Skorları: [ 1 23]
✅ China Mevcut -> Skorları: [14 28]
✅ Germany Mevcut -> Skorları: [ 5 44]
